## Notebook Setup 
The following cell will install Drake, checkout the underactuated repository, and set up the path (only if necessary).
- On Google's Colaboratory, this **will take approximately two minutes** on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  Colab will ask you to "Reset all runtimes"; say no to save yourself the reinstall.
- On Binder, the machines should already be provisioned by the time you can run this; it should return (almost) instantly.

More details are available [here](http://underactuated.mit.edu/drake.html).


In [ ]:
try:
    import pydrake
    import underactuated
except ImportError:
    !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
    from jupyter_setup import setup_underactuated
    setup_underactuated()

# Setup matplotlib.  
from IPython import get_ipython
if get_ipython() is not None: get_ipython().run_line_magic("matplotlib", "inline")

In [ ]:
# python libraries
import numpy as np
import matplotlib.pyplot as plt

# pydrake imports
from pydrake.all import DiagramBuilder, Variable, SymbolicVectorSystem, LogOutput, Simulator

# underactuated imports
from underactuated import plot_2d_phase_portrait

## Problem Description
In this problem you will write the dynamics equation for the Hopfield network.
At the end of the notebook, you will be able to check your work in test cases we set up for you.

**These are the main steps of the exercise:**
1. Write the function "dynamics" that defines the differential equation. _You need to write a piece of code to implement the system dynamics._

2. Modify the system dynamics to memorize MNIST data. _You need to write a piece of code to modify the matrix $A$._


### Question (a) - (c)

Write the function for the dynamical system defined in the quesiton. This function should return the right-hand side of the dynamical equation: $\dot{x} = A^T\,\text{softmax}(\beta\, A\, x) - x$. Write the function to output $A^T\,\text{softmax}(\beta\, A\, x) - x$ given the state $x$, matrix $A$, and scalar $\beta$.

In [4]:

def dynamics(x, A, beta):
    """outputs the right hand side of differential equation in Hopfield dynamical system.
    
    ARGUMENTS: x: numpy array of size n
               A: numpy array of size m by n
               beta: scalar
    RETURNS:   numpy array of size n
    """
    ################### Your solution goes here #######################
    
    return 
    
    ###################################################################



Here we create a Drake dynamical system using the class `SymbolicVectorSystem`.
This requires the dynamics to be written in the form $\dot{\mathbf{x}} = f(\mathbf{x})$, where $\mathbf{x}$ is a vector of Drake symbolic variables.
`SymbolicVectorSystem` is just one of the many options we have in Drake: when systems will get more complicated, writing the dynamics by hand can be rather tedious...

The cell below should plot the phase portrait if the function dynamics(x, A, beta) is implemented correctly.

In [ ]:
A = np.array([[1,0],[0,1],[-1,-1]])
beta = 5

# state variables
x1 = Variable("x1")
x2 = Variable("x2")
x = [x1, x2]

# Drake nonlinear system
system = SymbolicVectorSystem(state=x, output=x,  dynamics=dynamics(x, A, beta))

# initialize plot and make it big enough
plt.figure(figsize=(10, 10))

# plot the phase portrait of the 2d system
plot_2d_phase_portrait((lambda x: dynamics(x, A, beta)), x1lim=[-2, 2], x2lim=[-2, 2], linewidth=1, density=2)

### Question (d)
In this question, we will modify the constant matrix $A$ to store a subset of the MNIST dataset. The MNIST dataset contains 28 by 28 images of hand-written numbers. 

In the following cell, we load training data and corrupted images we want to restore. The variable `training_data` contains the four images that should be stored as the equilibriums to our dynamical system.

The variable `corrupted_img` contains a set of corrupted images that are used as the initial conditions.

In [ ]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

training_data = [x_train[0], x_train[1], x_train[4], x_train[6]]

corrupted_img = [x_train[8], x_train[34], x_train[5]]

for i, item in enumerate(corrupted_img):
    corrupted_img[i][14:,:]=0

print("Training Data")
fig, ax = plt.subplots(1, 4, sharex='col', sharey='row')
for i, item in enumerate(training_data):
    ax[i].imshow(item,cmap='gray')
plt.show()

print("Corrupted Images")
fig, ax = plt.subplots(1, 3, sharex='col', sharey='row')
for i, item in enumerate(corrupted_img):
    ax[i].imshow(item,cmap='gray')
plt.show()


Note that 

Modify the matrix $A$ to store the four MNIST training data provided as a list of images in . 

We will vectorize the image to the state ($x\in\mathbb{R}^{784}$) and store training data to the equilibriums of our dynamical system.

Modify the matrix $A\in\mathbb{R}^{4\times 784}$ to store four training data ($m=4$) in variable `training_data`.

Simulate the dynamical system to restore the image from the initial query provided in the notebook.


The data is currently provided as a 28 by 28 image ($\mathbb{R}^{28\times 28}$). First, modify the image data to a vector form, such that it is in the same format as the state $x\in\mathbb{R}^{784}$. Then, recall that the question (c) showed the relationship between the matrix $A$ and the equilibriums. Construct the matrix $A$ such that the dynamical system maps the training data set to the equilibrium.

In [ ]:

################### Your solution goes here ############################
# Assume that the transition from image (28 by 28 array) to state (784 by 1 array) is done by 
# x_state = x_image.reshape((-1))

def calculate_A(training_data):
    """outputs the right hand side of differential equation in Hopfield dynamical system.
    
    ARGUMENTS: training_data: a list of numpy array of arbitrary size (e.g., 28 by 28 for MNIST problem)
    RETURNS:   numpy array of size m by n (e.g., 4 by 784 for MNIST problem)
    """
    return []

#########################################################################

import pydrake
A = calculate_A(training_data)
beta = 1e-5

# state variables
x = pydrake.symbolic.MakeVectorContinuousVariable(784, 'x')
system = SymbolicVectorSystem(state=x, output=x,  dynamics=dynamics(x, A, beta))


## Drake Diagram
We then construct a Drake diagram.
This is nothing more than a set of interconnected dynamical systems (similiar to the Simulink idea, if you ever used it).
Our diagram will be very simple: we just connect our dynamical system to a logger, which will measure and store the system state during the simulation (similar to the Simulink `To Workspace` block).

In [ ]:
# initialize builder of the diagram
builder = DiagramBuilder()

# add our dynamical system
# (note: builder.AddSystem() returns a pointer to the system passed as input,
# hence it is safe to assign the name "system" to its output)
system = builder.AddSystem(system)

# logger block to measure and store the state
# connected to the (first and only) output port of the dynamical system
logger = LogOutput(system.get_output_port(0), builder)

# finalize diagram
diagram = builder.Build()

## Simulation
We are ready to simulate our dynamical system.
To this end we just feed our diagram in a the Drake `Simulator` and `AdvanceTo` the desired time.

In [ ]:
# function that given the initial state
# and a simulation time returns the system trajectory
def simulate(x, sim_time):
    
    # clean the logger from old trajectories
    logger.reset()
    
    # set up the simulator
    simulator = Simulator(diagram)
    
    # set initial conditions
    # (for now, think of "context" as a synonym of state)
    context = simulator.get_mutable_context()
    context.SetContinuousState(x)
    
    # simulate from t=0 to t=sim_time
    simulator.AdvanceTo(sim_time)
    
    # return the output (here = state) trajectory
    return logger.data()

We can visualize the trajectory by restoring the state format ($\mathbb{R}^{28\times 28}$) to an image format($\mathbb{R}^{784}$).

In [ ]:
# Visualize the simulated trajectories for corrupted image 1.

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import rc
rc('animation', html='jshtml')

x_init = corrupted_img[0].reshape((-1))
traj = simulate(x_init, 5)

fig = plt.figure(figsize=(6, 6))
im = plt.imshow(traj.T[0].reshape(28,28),cmap='gray')

def animate(i):    
    im.set_array(traj.T[i].reshape(28,28))
    return im,

# create animation using the animate() function
myAnimation = animation.FuncAnimation(fig, animate, frames=range(len(traj.T)), interval=10, blit=True, repeat=True)

plt.close()
myAnimation

In [ ]:
# Visualize the simulated trajectories for corrupted image 2.

x_init = corrupted_img[1].reshape((-1))
traj = simulate(x_init, 5)

fig = plt.figure(figsize=(6, 6))
im = plt.imshow(traj.T[0].reshape(28,28),cmap='gray')

def animate(i):    
    im.set_array(traj.T[i].reshape(28,28))
    return im,

# create animation using the animate() function
myAnimation = animation.FuncAnimation(fig, animate, frames=range(len(traj.T)), interval=10, blit=True, repeat=True)

plt.close()
myAnimation

In [ ]:
# Visualize the simulated trajectories for corrupted image 3.

x_init = corrupted_img[2].reshape((-1))
traj = simulate(x_init, 5)

fig = plt.figure(figsize=(6, 6))
im = plt.imshow(traj.T[0].reshape(28,28),cmap='gray')

def animate(i):    
    im.set_array(traj.T[i].reshape(28,28))
    return im,

# create animation using the animate() function
myAnimation = animation.FuncAnimation(fig, animate, frames=range(len(traj.T)), interval=10, blit=True, repeat=True)

plt.close()
myAnimation

## Fun Applications (Optional)

In this section, we explore fun applications of Hopfield networks. You do not need to write any additional code for this section.

For those of you who watched Pokemon, the viewers are asked to guess Pokemon names from its shadows.
(For example, https://www.youtube.com/watch?v=an2j2IGS7us&ab_channel=SasukeiKun)

We can also train Hopfield network to memorize Pokemons.

In [ ]:
from skimage import io
from skimage.transform import resize

urls = ["https://upload.wikimedia.org/wikipedia/en/5/59/Pok%C3%A9mon_Squirtle_art.png",
       "https://upload.wikimedia.org/wikipedia/en/a/a5/Pok%C3%A9mon_Charmander_art.png",
       "https://upload.wikimedia.org/wikipedia/en/2/28/Pok%C3%A9mon_Bulbasaur_art.png"]

training_data = []
query_data = []
for url in urls:
    resized_img = resize(io.imread(url), (80, 80))
    query_img = np.ones(resized_img.shape)
    query_img[:,:,0] = (resized_img[...,:3].max(axis=2)<0.01)*0.
    query_img[:,:,1] = (resized_img[...,:3].max(axis=2)<0.01)*0.
    query_img[:,:,2] = (resized_img[...,:3].max(axis=2)<0.01)*0.
    query_img[:,:,3] = resized_img[:,:,3]
    training_data.append(resized_img)
    query_data.append(query_img)

print("Training Data")
fig, ax = plt.subplots(1, 3, sharex='col', sharey='row')
for i, item in enumerate(training_data):
    ax[i].imshow(item)
plt.show()

print("Guess who?")
fig, ax = plt.subplots(1, 3, sharex='col', sharey='row')
for i, item in enumerate(query_data):
    ax[i].imshow(item)
plt.show()

In [ ]:
from pydrake.all import LeafSystem, BasicVector

class HopfieldNet(LeafSystem):
    def __init__(self, A, beta):
        LeafSystem.__init__(self)
        self.DeclareContinuousState(A.shape[1])
        self.DeclareVectorOutputPort(BasicVector(A.shape[1]), self.CopyStateOut)
        self.A = A
        self.beta = beta
        
    def DoCalcTimeDerivatives(self, context, derivatives):
        x = context.get_continuous_state_vector().CopyToVector()
        xdot = dynamics(x, self.A, self.beta)
        derivatives.get_mutable_vector().SetFromVector(xdot)
        
    def CopyStateOut(self, context, output):
        x = context.get_continuous_state_vector().get_value()
        y = output.get_mutable_value()
        y[:] = x

In [ ]:
A_img = []
for item in training_data:
    A_img += [item.reshape((-1))]
A = np.array(A_img)

beta = 1e-3

# initialize builder of the diagram
builder = DiagramBuilder()

# add our dynamical system
# (note: builder.AddSystem() returns a pointer to the system passed as input,
# hence it is safe to assign the name "system" to its output)
system = builder.AddSystem(HopfieldNet(A, beta))

# logger block to measure and store the state
# connected to the (first and only) output port of the dynamical system
logger = LogOutput(system.get_output_port(0), builder)

# finalize diagram
diagram = builder.Build()

In [ ]:
# function that given the initial state
# and a simulation time returns the system trajectory
def simulate(x, sim_time):
    
    # clean the logger from old trajectories
    logger.reset()
    
    # set up the simulator
    simulator = Simulator(diagram)
    
    # set initial conditions
    # (for now, think of "context" as a synonym of state)
    context = simulator.get_mutable_context()
    context.SetContinuousState(x)
    
    # simulate from t=0 to t=sim_time
    simulator.AdvanceTo(sim_time)
    
    # return the output (here = state) trajectory
    return logger.data()

In [ ]:
# Visualize the simulated trajectories.
x_init = query_data[0].reshape((-1))
traj = simulate(x_init, 5)

fig = plt.figure(figsize=(6, 6))
im = plt.imshow(traj.T[0].reshape(80, 80, 4))

def animate(i):    
    im.set_array(traj.T[i].reshape(80, 80, 4))
    return im,

#create animation using the animate() function
myAnimation = animation.FuncAnimation(fig, animate, frames=range(len(traj.T)), interval=50, blit=True, repeat=True)

plt.close()
myAnimation

While the Hopfield network successfully retrieved Squirtle, there are limitations to this particular network. Can you identify what it is and propose an explanation why it happens?

For example, try Charmander as the initial condition.

In [ ]:
# Visualize the simulated trajectories.
x_init = query_data[1].reshape((-1))
traj = simulate(x_init, 5)

fig = plt.figure(figsize=(6, 6))
im = plt.imshow(traj.T[0].reshape(80, 80, 4))

def animate(i):    
    im.set_array(traj.T[i].reshape(80, 80, 4))
    return im,

#create animation using the animate() function
myAnimation = animation.FuncAnimation(fig, animate, frames=range(len(traj.T)), interval=50, blit=True, repeat=True)

plt.close()
myAnimation